# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   Dataset dapat diperoleh dari berbagai sumber, seperti public repositories (*Kaggle*, *UCI ML Repository*, *Open Data*) atau data primer yang Anda kumpulkan sendiri.


# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning atau deep learning.

In [ ]:
#Type your code here
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

: 

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

Jika dataset berupa unstructured data, silakan sesuaikan dengan format seperti kelas Machine Learning Pengembangan atau Machine Learning Terapan

In [ ]:
#Type your code here
DATA_ROOT_PATH = '../data'
data_path = os.path.join(DATA_ROOT_PATH, 'Mobile.csv')

df = pd.read_csv(data_path)
df.head()

# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [ ]:
df.info()

## **Pembersihan Data Terinput**

Menghapus komponen teks pada setiap kolom yang seharusnya numerik. Lalu  mengubah ke tipe data yang benar.

In [ ]:
obj_cols = df.select_dtypes(include='object')

print('Kolom-kolom object:')
obj_cols.columns.tolist()

In [ ]:
col_with_digit = []

for col in obj_cols:
    # Memgambil sample data pada setiap kolom, baris pertama
    first_row = df[col].iloc[0]
    # Jika ditemukan data dengan nilai numerik didalamnya
    if re.search(r'\d+', first_row):
        # Simpan nama col
        col_with_digit.append(col)
        # Maka menyisakan nilai numerik itu, dan hapus teks non numerik
        df[col] = df[col].str.replace(r'\D+', '', regex=True)
    else:
        continue

### Hasil Pembersihan Data Terinput

In [ ]:
df[col_with_digit].head()

## Ubah Tipe Data

In [ ]:
df[col_with_digit] = df[col_with_digit].astype('float64')

## Visualisasi Distribusi Data

In [ ]:
#Type your code here
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

df[numeric_cols].hist(bins=20, figsize=(12, 8), grid=False)
plt.tight_layout()
plt.show()

In [ ]:
new_obj_cols = df.select_dtypes(include='object')

# Visualisasi kolom bertipe object menggunakan bar chart dan pie chart
for col in new_obj_cols.columns:
    plt.figure(figsize=(10, 4))
    # Bar chart
    plt.subplot(1, 2, 1)
    df[col].value_counts().plot(kind='bar', color='skyblue')
    plt.title(f'Bar Chart of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=45)

    # Pie chart
    plt.subplot(1, 2, 2)
    df[col].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=plt.cm.Paired.colors)
    plt.title(f'Pie Chart of {col}')
    plt.ylabel('')

    plt.tight_layout()
    plt.show()

# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning.

Jika Anda menggunakan data teks, data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang bisa dilakukan, tetapi **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

Cukup sesuaikan dengan karakteristik data yang kamu gunakan yah. Khususnya ketika kami menggunakan data tidak terstruktur.

## 1. Menghapus atau Menangani Data Kosong (Missing Values)

In [ ]:
# Cek data kosong
total_na = df.isna().sum()

def delete_na(total_na, df):
    if len(total_na) > 0:
        df = df.dropna(axis=0)

    return df

df = delete_na(total_na, df)

Tidak ada data kosong, maka tidak ada proses penghapusan data kosong

## 2. Menghapus Data Duplicate

In [ ]:
total_dups = df.duplicated().sum()

def delete_dups(total_dups, df):
    if total_dups > 0:
        df = df.drop_duplicates(axis=0)

    return df

Tidak ada data duplikat, maka tidak ada proses penghapusan data duplikasi

## 3. Encoding Fitur Objek

In [ ]:
remain_object_cols = df.select_dtypes(include='object')

for col in remain_object_cols.columns:
    print(f"Nilai unik Kolom {col}:")
    print("="*4)
    print(df[col].unique())
    print("="*4)

In [ ]:
# Proses Encoding
binary_cat = {'No': 0, 'Yes': 1}
target_cat = {"Low cost": 0, "Medium cost": 1,
              "High cost": 2, "Very High cost": 3}

encoders = {'Binary_Cat': binary_cat,
            'Target_Cat': target_cat}

target_col = 'price_range'

def encoding_object_feat(remain_obj_cols, target_col, all_cats):
    for_encode_cols = remain_obj_cols.columns
    
    for col in for_encode_cols:
        scaler = all_cats['Binary_Cat'] if col != target_col else all_cats['Target_Cat']
        df[col] = df[col].map(scaler)

    return df

df = encoding_object_feat(remain_object_cols, target_col, encoders)

In [ ]:
df.head()

## 4. Normalisasi atau Standarisasi Fitur

### Cek Korelasi Fitur

In [ ]:
corr_matrix = df.corr(numeric_only=True)

plt.figure(figsize=(12, 8))
sns.heatmap(
    corr_matrix,
    annot=True,     
    fmt=".2f",
    cmap="coolwarm",
    center=0,
    cbar=True
)
plt.title("Heatmap Korelasi Antar Fitur")
plt.show()

In [ ]:
feature_target_corr = df.corr(numeric_only=True)[[target_col]].drop(index=target_col)
feature_target_corr = feature_target_corr.sort_values(by=target_col, ascending=False)

feature_target_corr

### Standarisasi

In [ ]:
df[numeric_cols].describe()

**Standarisasi Kolom Range Spreading**

Ditandai dengan standar deviasi yang lebih tinggi dari rata-rata setiap fitur yang ada. Serta korelasi yang cukup merepresentasikan hubungan dengan fitur target.

In [ ]:
std_scaler = StandardScaler()

def scale_with_std_scaler(cols, df, std_scaler):
    df[cols] = std_scaler.fit_transform(df[cols])
    return df

In [ ]:
col_to_scale = ['Ram_mb', 'Battery_power_mAh', 'Pixel_width', 'px_height']

df = scale_with_std_scaler(col_to_scale, df, std_scaler)

In [ ]:
df.head()

## 5. Deteksi dan Penanganan Outliers

In [ ]:
num_cols = df.select_dtypes(exclude='object').columns

def visualize_outliers(n_cols: int = 3,
                       figsz_x: int = 15,
                       figsz_y: int = 5,
                       df: pd.DataFrame = df):

    n_cols = n_cols
    n_rows = int(np.ceil(len(num_cols) / n_cols))

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(figsz_x, figsz_y * n_rows))
    axes = axes.flatten()

    for i, col in enumerate(num_cols):
        sns.boxplot(y=df[col], ax=axes[i], color="skyblue")
        axes[i].set_title(f"Outlier Check: {col}")

    # hapus subplot kosong jika jumlah kolom < n_rows*n_cols
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

visualize_outliers()

In [ ]:
def remove_outliers_iqr(data: pd.DataFrame = df, cols: list = []) -> pd.DataFrame:
    cleaned_data = data.copy()
    for col in cols:
        Q1 = cleaned_data[col].quantile(0.25)
        Q3 = cleaned_data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        cleaned_data = cleaned_data[(cleaned_data[col] >= lower) & (cleaned_data[col] <= upper)]
    return cleaned_data

df_cleaned = remove_outliers_iqr(df, num_cols)

print(f"Sebelum hapus outliers: {df.shape}")
print(f"Setelah hapus outliers: {df_cleaned.shape}")

## 6. Reduksi Dimensi (Dengan LDA)

In [ ]:
X = df_cleaned.drop(columns=[target_col])
y = df_cleaned[target_col]

def reduction_with_lda(X: pd.Series, 
                       y: pd.Series,
                       n_comp: int = 3) -> pd.DataFrame:

    lda = LinearDiscriminantAnalysis(n_components=n_comp)
    X_lda = lda.fit_transform(X, y)

    return X_lda

In [ ]:
X_lda = reduction_with_lda(X, y, 3)

plt.figure(figsize=(8,6))
for label in y.unique():
    plt.scatter(
        X_lda[y == label, 0],
        X_lda[y == label, 1],
        label=f"Class {label}",
        alpha=0.7
    )

plt.xlabel("LD1")
plt.ylabel("LD2")
plt.title("LDA: Proyeksi ke 2 Dimensi")
plt.legend()
plt.show()

## Modelling

### Data Splitting

In [ ]:
def data_splitting(X: pd.Series,
                   y: pd.Series,
                   rand_state: int = 42,
                   test_size: float = 0.2) -> pd.Series:
    X_train, y_train, X_test, y_test = train_test_split(
        X,
        y,
        random_state=rand_state,
        test_size=test_size
    )

    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = data_splitting(X_lda, y)

In [ ]:
model = SVC(
    C=10,
    gamma=0.1,
    kernel='rbf',
    random_state=42
)

model.fit(X_train, y_train)

predicted_qual = model.predict(X_test)
predicted_qual